# 라이브러리 다운

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install datasets

In [ ]:
pip install evaluate

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
from transformers import RobertaForSequenceClassification
import pandas as pd
from datasets import Dataset
import torch
from transformers import DataCollatorWithPadding, Trainer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 32)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should pr

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.text = df['Title'].values
        self.labels = df['Genre'].values
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [ ]:
df = pd.read_csv('vector_data.csv',encoding = "ISO-8859-1")
data = df[['Title', 'Genre']]

data.to_csv('vetor_data_1.csv',index = False, encoding = "ISO-8859-1")

In [ ]:
num = len(data)
train_df = data.iloc[:int(num*0.75), :]
eval_df = data.iloc[int(num*0.75):, :]

train_dataset = MyDataset(
    train_df,
    tokenizer,
    max_length=128
)

eval_dataset = MyDataset(
    eval_df,
    tokenizer,
    max_length=128
)



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after eachh epoch
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3      )

In [ ]:
# from sklearn.metrics import accuracy_score

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return {'accuracy': accuracy_score(labels, predictions)}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
# )


In [ ]:
# trainer.train()

In [ ]:
from torch.utils.data import DataLoader

class MyTrainer(Trainer):
    def get_train_dataloader(self):
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        data_loader = DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )
        return data_loader

    def get_eval_dataloader(self, eval_dataset=None):
        if eval_dataset is None and self.eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        data_loader = DataLoader(
            eval_dataset,
            batch_size=self.args.eval_batch_size,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )
        return data_loader


In [ ]:
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: ignored

# 재시도

In [ ]:
df.head(5)

,Unnamed: 0,Title,Genre
0,0,Doug the Pug 2016 Wall Calendar,3
1,1,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",3
2,2,365 Cats Color Page-A-Day Calendar 2016,3
3,3,Sierra Club Engagement Calendar 2016,3
4,4,Sierra Club Wilderness Calendar 2016,3


In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
from transformers import AutoTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer

# Read the dataset
df = pd.read_csv('/content/vetor_data_1.csv', encoding="ISO-8859-1")

# Genre 데이터의 범주를 정수 인덱스로 변환
label_encoder = LabelEncoder()
df['Genre'] = label_encoder.fit_transform(df['Genre'])

# Convert the DataFrame into a Hugging Face Dataset
data = Dataset.from_pandas(df)

# Tokenizer initialization
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["Title"], padding="max_length", truncation=True)

# Split dataset into train and test
raw_datasets = data.train_test_split(test_size=0.25)

# Map the tokenization function to the title text (it will also remove the column 'Title' and replace it with the new ones)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=["Title"])

# Model initialization
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=32)

# Training arguments
training_args = TrainingArguments(
    "test_trainer",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Override the compute_loss method of the Trainer
def compute_loss(model, inputs):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(logits, labels)
    return loss

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,

)
trainer.compute_loss = compute_loss
trainer.train()


Map:   0%|          | 0/155678 [00:00<?, ? examples/s]

Map:   0%|          | 0/51893 [00:00<?, ? examples/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should pr

KeyError: ignored

In [ ]:
pip install transformers[torch]